In [1]:
import os
import datetime
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
if 'current_dir' in locals():
    print("Directory has been set before:")
else:
    current_dir = os.getcwd()[:-9] 
print(current_dir)
os.chdir(current_dir + '/scripts')
import preprocessing
os.chdir(current_dir)
submission = pd.read_csv(current_dir + 'submissions/submission_v1.csv') #submission as template (version is not important)

/home/harry/Documents/customer-loyalty/


In [2]:
train_x, train_l, testset_x = preprocessing.load_data()

/home/harry/Documents/customer-loyalty/scripts
this is a test feature : test_feature_fam.csv
this is a train feature : train_feature_fam.csv
this is a train feature : train_feature_avg_purchase.csv


## Training
Training will be done in the following cells. First a random part of the training data is assigned as training set and the leftovers will be the validation set. By making the _trainsize_ variable round(train.shape[0]/1), the entirety of the training data is used as training set, so no evaluation will be done in the section _Testing within training set_.

In [3]:
# First XGBoost model for Pima Indians dataset
from numpy import loadtxt
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [4]:
model = XGBClassifier()

In [ ]:
model.fit(train_x, train_l)

## Predictions for test set (creating a submission .csv file)
Make sure to use the entire training set in the the "Training" section

In [ ]:
testset = np.array(testdata)
testset_x = testset[:,2:features+2]
# make predictions for test data
y_pred = model.predict(testset_x)
predictions = [round(value) for value in y_pred]
submission.target = predictions
# # evaluate predictions
# accuracy = accuracy_score(y_test, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
# submission.target = submission_predictions

for i in range(1,100,1):
    if os.path.isfile('submissions/submission_v0-00' + str(i) + '.csv') == False:
        submission.to_csv('submissions/submission_v0-00' + str(i) + '.csv', index = False)
        print('Submission was saved under the name:' + '\033[1m' ' submission_v0-00' + str(i) + '.csv' + '\033[0;0m')
        break

# [Errno 13] Permission denied: 'submissions/submission_v0-001.csv'
#        means that the file is still opened somewhere

In [ ]:
# run this after previous cel to see the submission
submission